In [48]:
!pip install SimpleITK

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [49]:
import SimpleITK as sitk
import numpy as np
import torch

In [53]:
inputImageFileName = "case2_T50-ssm.mha"
maskImageName = "case2_T50-ssm_mask.mha"

In [54]:
shape = [112, 256, 256]
spacing = [1.16, 1.16, 2.5]
direction = [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, -1.0]

In [55]:
def segment(input_image, output_mask=maskImageName, radius=(12, 12, 12),return_sitk=False):
    image = sitk.InvertIntensity(sitk.Cast(sitk.ReadImage(input_image),sitk.sitkFloat32))
    mask = sitk.OtsuThreshold(image)
    dil_mask = sitk.BinaryDilate(mask, (0, 0, 0))
    component_image = sitk.ConnectedComponent(dil_mask)
    sorted_component_image = sitk.RelabelComponent(component_image, sortByObjectSize=True)
    largest_component_binary_image = sorted_component_image == 1
    mask_closed = sitk.BinaryMorphologicalClosing(largest_component_binary_image, (12, 12, 12))
    dilated_mask = sitk.BinaryDilate(mask_closed, (0, 0, 0))
    filled_mask = sitk.BinaryFillhole(dilated_mask)
    if return_sitk:
        return filled_mask
    else:
        sitk.WriteImage(filled_mask, output_mask)

In [56]:
segment(inputImageFileName)